In [ ]:
from tkinter import *

import os
import cv2
import numpy as np
from PIL import ImageGrab
from keras.models import load_model
from tkinter import filedialog as fd
from tkinter.messagebox import showinfo
from tkinter import ttk
import time
import psutil
from tkinter import messagebox
from PIL import Image
import pytesseract
import io
import tkinter.font as font


user_list = psutil.users()
for user in user_list:
    username = user.name

model = load_model('mnist.h5')
image_folder = f"C:\\Users\\{username}\\"

root = Tk()
root.geometry('900x900')
root.resizable(0, 0)
root.title("Digit & Text Recognition By Erdeniz Unvan")

lastx, lasty = None, None
image_number = 0

cv = Canvas(root, width=900, height=600, bg='white' )
cv.grid(row=0, column=0, pady=2, columnspan=2)


def clear_widget():
    global cv
    cv.delete('all')


def draw_lines(event):
    global lastx, lasty
    x, y = event.x, event.y
    cv.create_line((lastx, lasty, x, y), width=8, fill='black', capstyle=ROUND, smooth=TRUE, splinesteps=12)
    lastx, lasty = x, y


def activate_event(event):
    global lastx, lasty
    cv.bind('<B1-Motion>', draw_lines)
    lastx, lasty = event.x, event.y


cv.bind('<Button-1>', activate_event)


def convert_to_text():
    filetypes = (
        ('JPG files', '*.jpg'),
        ('PNG files', '*.png')
    )

    filename = fd.askopenfilename(
        title='Open a file',
        initialdir='/',
        filetypes=filetypes)

    showinfo(
        title='Selected File',
        message=filename
    )
    basename = os.path.basename(filename)
    pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files (x86)\Tesseract-OCR\tesseract.exe"
    #language_options=['English','German','French','Italian','Spanish','Turkish','Romanian', 'Ukrainian']
    #eng,deu, fra, ita, spa, tur, ukr, ron
    
    language = language_combo.get()
    if language=='English':
        language=language.replace('English','eng')
    elif language=='German':
        language=language.replace('German','deu')
    elif language=='French':
        language=language.replace('French','fra')
    elif language=='Italian':
        language=language.replace('Italian','ita')
    elif language=='Spanish':
        language=language.replace('Spanish','spa')
    elif language=='Turkish':
        language=language.replace('Turkish','tur')
    elif language=='Ukrainian':
        language=language.replace('Ukrainian','ukr')
    elif language=='Romanian':
        language=language.replace('Romanian','ron')
    
    a=pytesseract.image_to_string(Image.open(basename),lang=language)
    response = messagebox.askyesno('Press Yes To Save The File Or No To Quit',a)
    if response == True:
        file=f'{basename}.txt'
        with open(file,'w',encoding='utf-8') as f:
            f.write(a)
        messagebox.showinfo('The File is saved',f'The file of {file} is saved into your directory of {os.getcwd()}')
    else:
        messagebox.showinfo('The File is not saved',f'The file is not saved')

def select_file():
    filetypes = (
        ('JPG files', '*.jpg'),
        ('PNG files', '*.png')
    )

    filename = fd.askopenfilename(
        title='Open a file',
        initialdir='/',
        filetypes=filetypes)

    showinfo(
        title='Selected File',
        message=filename
    )
    basename = os.path.basename(filename)
    originalImage = cv2.imread(basename)
    grayImage = cv2.cvtColor(originalImage, cv2.COLOR_BGR2GRAY)
    (thresh, blackAndWhiteImage) = cv2.threshold(grayImage, 127, 255, cv2.THRESH_BINARY)
    cv2.imwrite('1siyah_beyaz.png',blackAndWhiteImage)

    img1 = cv2.imread('1siyah_beyaz.png')
    rsz_img = cv2.resize(img1, None, fx=0.25, fy=0.25) # resize since image is huge
    gray = cv2.cvtColor(rsz_img, cv2.COLOR_BGR2GRAY) # convert to grayscale
    retval, thresh_gray = cv2.threshold(gray, thresh=100, maxval=255, type=cv2.THRESH_BINARY)
    points = np.argwhere(thresh_gray==0) # find where the black pixels are
    points = np.fliplr(points) # store them in x,y coordinates instead of row,col indices
    x, y, w, h = cv2.boundingRect(points) # create a rectangle around those points
    x, y, w, h = x-10, y-10, w+20, h+20 # make the box a little bigger
    crop = gray[y:y+h, x:x+w] # create a cropped region of the gray image
    retval, thresh_crop = cv2.threshold(crop, thresh=200, maxval=255, type=cv2.THRESH_BINARY)
    cv2.imwrite('2siyah_beyaz_crop.png',blackAndWhiteImage)
    
    img = Image.open("2siyah_beyaz_crop.png")
    colors=[]
    width = img.size[0]
    height = img.size[1]
    for i in range(0,width):# process all pixels
        for j in range(0,height):
            data = img.getpixel((i,j))
            colors.append(data)
    white_colors = colors.count(255)
    other_colors= len(colors)-white_colors
    if other_colors > white_colors:
        img2 = Image.open("2siyah_beyaz_crop.png")
        img2 = img2.convert("RGB")
        datas = img2.getdata()
        new_image_data = []
        for item in datas:
            # change all white (also shades of whites) pixels to yellow
            if item[0] in list(range(0, 255)):
                new_image_data.append((255, 255, 255))
            else:
                new_image_data.append((0, 0, 0))
        # update image data
        img2.putdata(new_image_data)
        # save new image
        img2.save("3siyah_beyaz_crop_edit.png")
    elif other_colors < white_colors:
        img3 = Image.open("2siyah_beyaz_crop.png")
        img3.save("3siyah_beyaz_crop_edit.png")
    
    image = cv2.imread('3siyah_beyaz_crop_edit.png')
    gray = cv2.cvtColor(image.copy(), cv2.COLOR_BGR2GRAY)
    ret, th = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    contours = cv2.findContours(th, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0]

    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        # make a rectangle box around each curve
        cv2.rectangle(image, (x, y), (x + w, y + h), (255, 0, 0), 1)

        # Cropping out the digit from the image corresponding to the current contours in the for loop
        digit = th[y:y + h, x:x + w]

        # Resizing that digit to (18, 18)
        resized_digit = cv2.resize(digit, (18, 18))

        # Padding the digit with 5 pixels of black color (zeros) in each side to finally produce the image of (28, 28)
        padded_digit = np.pad(resized_digit, ((5, 5), (5, 5)), "constant", constant_values=0)

        digit = padded_digit.reshape(1, 28, 28, 1)
        digit = digit / 255.0

        pred = model.predict([digit])[0]
        final_pred = np.argmax(pred)
        data = str(final_pred) + ' ' + str(int(max(pred) * 100)) + '%'
        font = cv2.FONT_HERSHEY_SIMPLEX
        fontScale = 0.5
        color = (255, 0, 0)
        thickness = 1
        cv2.putText(image, data, (x, y - 5), font, fontScale, color, thickness)

    cv2.imshow('Predictions', image)
    cv2.waitKey(0)

def Recognize_Digit():
    global image_number
    filename = f"{time.ctime()[11:19].replace(':','_')}.png"
    widget = cv

    x = root.winfo_rootx() + widget.winfo_rootx()
    y = root.winfo_rooty() + widget.winfo_rooty()
    x1 = x + widget.winfo_width()
    y1 = y + widget.winfo_height()
    print(x, y, x1, y1)
        # get image and save
    ImageGrab.grab().crop((x, y, x1, y1)).save(image_folder + filename)
    image = cv2.imread(image_folder + filename, cv2.IMREAD_COLOR)
    gray = cv2.cvtColor(image.copy(), cv2.COLOR_BGR2GRAY)
    ret, th = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    contours = cv2.findContours(th, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0]

    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        # make a rectangle box around each curve
        cv2.rectangle(image, (x, y), (x + w, y + h), (255, 0, 0), 1)

        # Cropping out the digit from the image corresponding to the current contours in the for loop
        digit = th[y:y + h, x:x + w]

        # Resizing that digit to (18, 18)
        resized_digit = cv2.resize(digit, (18, 18))

        # Padding the digit with 5 pixels of black color (zeros) in each side to finally produce the image of (28, 28)
        padded_digit = np.pad(resized_digit, ((5, 5), (5, 5)), "constant", constant_values=0)

        digit = padded_digit.reshape(1, 28, 28, 1)
        digit = digit / 255.0

        pred = model.predict([digit])[0]
        final_pred = np.argmax(pred)

        data = str(final_pred) + ' ' + str(int(max(pred) * 100)) + '%'

        font = cv2.FONT_HERSHEY_SIMPLEX
        fontScale = 0.5
        color = (255, 0, 0)
        thickness = 1
        cv2.putText(image, data, (x, y - 5), font, fontScale, color, thickness)

    cv2.imshow('Predictions', image)
    cv2.waitKey(0)

btn_save_label = Label(root,text='Press this button after drawing number/numbers for multiple digit detection =>',width=62, height=2)
btn_save_label.grid(row=1, column=0,pady=1,padx=1)
btn_save = Button(text='Draw Numbers At The Board\nFor Recognizing Digits',width=30, height=2, command=Recognize_Digit)
btn_save.grid(row=1, column=1,pady=1,padx=1)
button_clear_label = Label(root,text='Press this button for deleting everything at the screen =>',width=62, height=2)
button_clear_label.grid(row=2, column=0,pady=1,padx=1)
button_clear = Button(text='Clear The Board',width=30, height=1, command=clear_widget)
button_clear.grid(row=2, column=1,pady=1, padx=1)
open_button_label = Label(root,text='Press this button for choosing a jpg/png file for the multiple digit detection =>',width=80, height=2)
open_button_label.grid(row=3, column=0,pady=1,padx=1)
open_button = Button(root,text='Choose A Photo\nFor Dectecting Digits',width=30, height=2,command=select_file)
open_button.grid(row=3, column=1,pady=1, padx=1)

language_options=['English','German','French','Italian','Spanish','Turkish','Romanian', 'Ukrainian']
category_label=Label(root, text = 'Select Language For Converting Photo To Text')
category_label.grid(row=4, column=0)
helv36 = font.Font(family='Helvetica',size=8 )
language_combo = ttk.Combobox(root, value=language_options,font=helv36,width=8)
language_combo.current(0)
language_combo.grid(row=4, column=1)

open_button_label2 = Label(root,text='Press this button for choosing a jpg/png file for converting to text =>',width=80, height=2)
open_button_label2.grid(row=5, column=0,padx=1)
open_button2 = Button(root,text='Choose A Photo For\n Converting To The Text',width=30, height=2,command=convert_to_text)
open_button2.grid(row=5, column=1)



#eng,deu, fra, ita, spa, tur, ukr, ron

root.mainloop()


224 64 1128 668
